In [1]:
import aoc
import cmath
import collections
from collections import defaultdict
from dataclasses import dataclass
from functools import cache, reduce
import itertools as it
import math
import more_itertools as mit
import networkx as nx
import numpy as np
import operator as op
import pandas as pd
import re
import statistics
import utils
import scipy

In [2]:
test_input="""--- scanner 0 ---
404,-588,-901
528,-643,409
-838,591,734
390,-675,-793
-537,-823,-458
-485,-357,347
-345,-311,381
-661,-816,-575
-876,649,763
-618,-824,-621
553,345,-567
474,580,667
-447,-329,318
-584,868,-557
544,-627,-890
564,392,-477
455,729,728
-892,524,684
-689,845,-530
423,-701,434
7,-33,-71
630,319,-379
443,580,662
-789,900,-551
459,-707,401

--- scanner 1 ---
686,422,578
605,423,415
515,917,-361
-336,658,858
95,138,22
-476,619,847
-340,-569,-846
567,-361,727
-460,603,-452
669,-402,600
729,430,532
-500,-761,534
-322,571,750
-466,-666,-811
-429,-592,574
-355,545,-477
703,-491,-529
-328,-685,520
413,935,-424
-391,539,-444
586,-435,557
-364,-763,-893
807,-499,-711
755,-354,-619
553,889,-390

--- scanner 2 ---
649,640,665
682,-795,504
-784,533,-524
-644,584,-595
-588,-843,648
-30,6,44
-674,560,763
500,723,-460
609,671,-379
-555,-800,653
-675,-892,-343
697,-426,-610
578,704,681
493,664,-388
-671,-858,530
-667,343,800
571,-461,-707
-138,-166,112
-889,563,-600
646,-828,498
640,759,510
-630,509,768
-681,-892,-333
673,-379,-804
-742,-814,-386
577,-820,562

--- scanner 3 ---
-589,542,597
605,-692,669
-500,565,-823
-660,373,557
-458,-679,-417
-488,449,543
-626,468,-788
338,-750,-386
528,-832,-391
562,-778,733
-938,-730,414
543,643,-506
-524,371,-870
407,773,750
-104,29,83
378,-903,-323
-778,-728,485
426,699,580
-438,-605,-362
-469,-447,-387
509,732,623
647,635,-688
-868,-804,481
614,-800,639
595,780,-596

--- scanner 4 ---
727,592,562
-293,-554,779
441,611,-461
-714,465,-776
-743,427,-804
-660,-479,-426
832,-632,460
927,-485,-438
408,393,-506
466,436,-512
110,16,151
-258,-428,682
-393,719,612
-211,-452,876
808,-476,-593
-575,615,604
-485,667,467
-680,325,-822
-627,-443,-432
872,-547,-609
833,512,582
807,604,487
839,-516,451
891,-625,532
-652,-548,-490
30,-46,-14"""

# Part 1

In [6]:
from scipy.spatial.distance import pdist
a = np.random.rand(3, 3)
b = a + (1, 1, 1)
print(find_match(a, b))
print(pdist(a))

(None, None)
[0.58208785 0.71540988 0.27118007]


In [21]:
# run to solve part 1
from collections import Counter
from scipy.spatial.distance import pdist

roz= np.array([
    [0, -1, 0],
    [1, 0, 0],
    [0, 0, 1]
])

rox= np.array([
    [1, 0, 0],
    [0, 0, -1],
    [0, 1, 0]
])

roy= np.array([
    [0, 0, 1],
    [0, 1, 0],
    [-1, 0, 0]
])

def get_all_rots(scanner):
    rots = [scanner]
    rots.append(rots[-1] @ rox)
    rots.append(rots[-1] @ rox)
    rots.append(rots[-1] @ rox)
    rots.append(rots[-1] @ roy)
    rots.append(rots[-1] @ rox)
    rots.append(rots[-1] @ rox)
    rots.append(rots[-1] @ rox)
    rots.append(rots[-1] @ roy)
    rots.append(rots[-1] @ rox)
    rots.append(rots[-1] @ rox)
    rots.append(rots[-1] @ rox)
    rots.append(rots[-1] @ roz @ roz)
    rots.append(rots[-1] @ rox)
    rots.append(rots[-1] @ rox)
    rots.append(rots[-1] @ rox)
    rots.append(rots[-1] @ roy)
    rots.append(rots[-1] @ rox)
    rots.append(rots[-1] @ rox)
    rots.append(rots[-1] @ rox)
    rots.append(rots[-1] @ roy)
    rots.append(rots[-1] @ rox)
    rots.append(rots[-1] @ rox)
    rots.append(rots[-1] @ rox)
    return rots

def parse_scanner(scanner_section):
    coors = re.findall(r"(-?\d+),(-?\d+),(-?\d+)", scanner_section)
    return np.array([(int(x), int(y), int(z)) for x, y, z in coors])

def find_match(existing, new):
    for e, n in it.product(existing, new):
        candidate_new = new + (e - n)
        if is_match(existing, candidate_new):
            return candidate_new, (e - n)
    return None, None

def is_match(a, b):
    aset = frozenset((x, y, z) for x, y, z in a)
    bset = frozenset((x, y, z) for x, y, z in b)
    return len(aset & bset) >= 12

def count_matching_dists(a, b):
    return sum(1 for x in a for y in b if x == y)

def part_1(aoc_input):
    scanner_sections = aoc_input.split('\n\n')
    scanners = [parse_scanner(section) for section in scanner_sections]
    rot_scanners = [get_all_rots(s) for s in scanners]
    distances = [pdist(s) for s in scanners]
    matching_dists = defaultdict(list)
    for i,j in it.product(range(len(scanners)), repeat=2):
        if i == j:
            continue
        if count_matching_dists(distances[i], distances[j]) >= 66:
            matching_dists[i].append(j)
    print(matching_dists)
        
    placed_scanners = [scanners[0]]
    completed_i = [0]
    offsets = [np.zeros(3)] + [None] * (len(scanners) - 1)
    
    while len(placed_scanners) < len(scanners):
        for j, placed_scan in zip(completed_i, placed_scanners):
            for i in matching_dists[j]:
                if i in completed_i:
                    continue
                all_rots = rot_scanners[i]
                for rot in all_rots:
                    match, rel_offset = find_match(placed_scan, rot)
                    if match is not None:
                        print(f"Found match: scanner {i} matches with placed scanner {j}, relative_offset {rel_offset}")
                        placed_scanners.append(match)
                        completed_i.append(i)
                        offsets[i] = offsets[j] + rel_offset
                        break
                else:
                    continue
                break
            else:
                continue
            break
        print("Iteration done")
    
    beacons = set()
    for p in placed_scanners:
        beacons.update((x, y, z) for x, y, z in p)
    return len(beacons)

part_1_test_answer = part_1(test_input)
print("TEST RESULT: ", part_1_test_answer)
part_1_answer = part_1(aoc.get_input(2021, 19))
print("ACTUAL RESULT: ", part_1_answer)

defaultdict(<class 'list'>, {0: [1], 1: [0, 3, 4], 2: [4], 3: [1], 4: [1, 2]})
Found match: scanner 1 matches with placed scanner 0, relative_offset [   68 -1246   -43]
Iteration done
Found match: scanner 3 matches with placed scanner 1, relative_offset [  -92 -2380   -20]
Iteration done
Found match: scanner 4 matches with placed scanner 1, relative_offset [  -20 -1133  1061]
Iteration done
Found match: scanner 2 matches with placed scanner 4, relative_offset [ 1105 -1205  1229]
Iteration done
TEST RESULT:  79
defaultdict(<class 'list'>, {0: [22], 1: [3, 7, 14, 15, 27], 2: [13], 3: [1], 4: [10], 5: [8, 12, 15], 6: [7, 15], 7: [1, 6, 16, 25], 8: [5, 24], 9: [17], 10: [4, 15, 27], 11: [21], 12: [5], 13: [2, 19, 25], 14: [1, 17, 23], 15: [1, 5, 6, 10], 16: [7, 27], 17: [9, 14], 18: [23], 19: [13, 21], 20: [23], 21: [11, 19, 25], 22: [0, 24, 26], 23: [14, 18, 20], 24: [8, 22], 25: [7, 13, 21], 26: [22], 27: [1, 10, 16]})
Found match: scanner 22 matches with placed scanner 0, relative_offse

In [139]:
# run to submit part 1

aoc.submit_answer(2021, 19, 1, part_1_answer)

Submitting for YEAR 2021, DAY 19, LEVEL 1: 362... Correct!


# Part 2

In [22]:
# run to solve part 2

def part_2(aoc_input):
    scanner_sections = aoc_input.split('\n\n')
    scanners = [parse_scanner(section) for section in scanner_sections]
    rot_scanners = [get_all_rots(s) for s in scanners]
    distances = [pdist(s) for s in scanners]
    matching_dists = defaultdict(list)
    for i,j in it.permutations(range(len(scanners)), 2):
        if count_matching_dists(distances[i], distances[j]) >= 66:
            matching_dists[i].append(j)
    print(matching_dists)
        
    placed_scanners = [scanners[0]]
    completed_i = [0]
    offsets = [np.zeros(3)] + [None] * (len(scanners) - 1)
    
    while len(placed_scanners) < len(scanners):
        for j, placed_scan in zip(completed_i, placed_scanners):
            for i in matching_dists[j]:
                if i in completed_i:
                    continue
                all_rots = rot_scanners[i]
                for rot in all_rots:
                    match, actual_offset = find_match(placed_scan, rot)
                    if match is not None:
                        print(f"Found match: scanner {i} matches with placed scanner {j}, actual_offset {actual_offset}")
                        placed_scanners.append(match)
                        completed_i.append(i)
                        offsets[i] = actual_offset
                        break
                else:
                    continue
                break
            else:
                continue
            break
        print("Iteration done")
    return int(max(pdist(offsets, 'cityblock')))

part_2_test_answer = part_2(test_input)
print("TEST RESULT: ", part_2_test_answer)
part_2_answer = part_2(aoc.get_input(2021, 19))
print("ACTUAL RESULT: ", part_2_answer)

defaultdict(<class 'list'>, {0: [1], 1: [0, 3, 4], 2: [4], 3: [1], 4: [1, 2]})
Found match: scanner 1 matches with placed scanner 0, actual_offset [   68 -1246   -43]
Iteration done
Found match: scanner 3 matches with placed scanner 1, actual_offset [  -92 -2380   -20]
Iteration done
Found match: scanner 4 matches with placed scanner 1, actual_offset [  -20 -1133  1061]
Iteration done
Found match: scanner 2 matches with placed scanner 4, actual_offset [ 1105 -1205  1229]
Iteration done
TEST RESULT:  3621
defaultdict(<class 'list'>, {0: [22], 1: [3, 7, 14, 15, 27], 2: [13], 3: [1], 4: [10], 5: [8, 12, 15], 6: [7, 15], 7: [1, 6, 16, 25], 8: [5, 24], 9: [17], 10: [4, 15, 27], 11: [21], 12: [5], 13: [2, 19, 25], 14: [1, 17, 23], 15: [1, 5, 6, 10], 16: [7, 27], 17: [9, 14], 18: [23], 19: [13, 21], 20: [23], 21: [11, 19, 25], 22: [0, 24, 26], 23: [14, 18, 20], 24: [8, 22], 25: [7, 13, 21], 26: [22], 27: [1, 10, 16]})
Found match: scanner 22 matches with placed scanner 0, actual_offset [  -63

In [157]:
# run to submit part 2

aoc.submit_answer(2021, 19, 2, part_2_answer)

Submitting for YEAR 2021, DAY 19, LEVEL 2: 12204... Correct!


In [8]:
x = [(0, 8), (0, 11), (0, 17), (1, 7), (1, 21), (1, 23), (2, 8), (2, 11), (2, 13), (2, 23), (3, 15), (4, 16), (4, 26), (4, 31), (4, 32), (5, 7), (5, 29), (6, 16), (6, 23), (8, 12), (8, 18), (8, 24), (9, 18), (9, 27), (10, 31), (10, 32), (12, 16), (12, 21), (12, 23), (13, 18), (14, 22), (14, 25), (14, 32), (15, 31), (16, 24), (17, 18), (17, 27), (19, 29), (20, 33), (21, 34), (22, 33), (25, 30), (26, 28)]



d = defaultdict(list)
for a, b in x:
    d[a].append(b)
    d[b].append(a)
d2 = {}
for k in sorted(d.keys()):
    d2[k] = d[k]
print(d2)
g = nx.Graph()
for k, v in d2.items():
    for vi in v:
        g.add_edge(k, vi)
len(list(nx.connected_components(g)))

{0: [8, 11, 17], 1: [7, 21, 23], 2: [8, 11, 13, 23], 3: [15], 4: [16, 26, 31, 32], 5: [7, 29], 6: [16, 23], 7: [1, 5], 8: [0, 2, 12, 18, 24], 9: [18, 27], 10: [31, 32], 11: [0, 2], 12: [8, 16, 21, 23], 13: [2, 18], 14: [22, 25, 32], 15: [3, 31], 16: [4, 6, 12, 24], 17: [0, 18, 27], 18: [8, 9, 13, 17], 19: [29], 20: [33], 21: [1, 12, 34], 22: [14, 33], 23: [1, 2, 6, 12], 24: [8, 16], 25: [14, 30], 26: [4, 28], 27: [9, 17], 28: [26], 29: [5, 19], 30: [25], 31: [4, 10, 15], 32: [4, 10, 14], 33: [20, 22], 34: [21]}


1

In [20]:
y = np.array([(-1061, -20, -1133), (-92, 2380, -20), (0, 0, 0), (68, 1246, -43), (1105, -1229, 1205)])
x = np.array([(-20, -1133, 1061), (-92, -2380, -20), (0, 0, 0), (68, -1246, -43), (1105, -1205, 1229)])
rotated_x = get_all_rots(x)
for a in rotated_x:
    print(np.all(a == y))

False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
